In [1]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [3]:
file_path = 'customer_churn_dataset-testing-master.csv'
df = pd.read_csv(file_path)

# Drop the 'CustomerID' and 'Churn' columns (as 'CustomerID' is not a feature and 'Churn' is the label)
features = df.drop(columns=['CustomerID', 'Churn'])
labels = df['Churn']

# Identify categorical columns
categorical_columns = ['Gender', 'Subscription Type', 'Contract Length']

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply the encoder to the categorical columns
encoded_features = encoder.fit_transform(features[categorical_columns])

# Convert the encoded features back into a DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the encoded columns with the original dataset (dropping the original categorical columns)
features_encoded = pd.concat([features.drop(columns=categorical_columns).reset_index(drop=True), encoded_df], axis=1)

# Split the dataset into training and testing sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(features_encoded, labels, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

# Reshape y_train and y_test for PyTorch
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)

In [4]:
# Define the neural network with one hidden layer
class ChurnPredictorOneHiddenLayer(nn.Module):
    def __init__(self):
        super(ChurnPredictorOneHiddenLayer, self).__init__()
        self.hidden = nn.Linear(X_train_tensor.shape[1], 4096)  # Hidden layer
        self.output = nn.Linear(4096, 1)  # Output layer

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = torch.sigmoid(self.output(x))
        return x

# Initialize the model, loss function, and optimizer
model_one_layer = ChurnPredictorOneHiddenLayer().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_one_layer.parameters(), lr=0.001)

# Train the model with 100 epochs
num_epochs = 100
for epoch in range(num_epochs):
    model_one_layer.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model_one_layer(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.3772
Epoch [20/100], Loss: 0.3461
Epoch [30/100], Loss: 0.3130
Epoch [40/100], Loss: 0.2909
Epoch [50/100], Loss: 0.2751
Epoch [60/100], Loss: 0.2603
Epoch [70/100], Loss: 0.2485
Epoch [80/100], Loss: 0.2387
Epoch [90/100], Loss: 0.2306
Epoch [100/100], Loss: 0.2233


In [5]:
# Define the neural network with two hidden layers
class ChurnPredictorTwoHiddenLayers(nn.Module):
    def __init__(self):
        super(ChurnPredictorTwoHiddenLayers, self).__init__()
        self.hidden1 = nn.Linear(X_train_tensor.shape[1], 4096)  # First hidden layer
        self.hidden2 = nn.Linear(4096, 64)  # Second hidden layer
        self.output = nn.Linear(64, 1)  # Output layer

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

# Initialize the model, loss function, and optimizer
model_two_layers = ChurnPredictorTwoHiddenLayers().to(device)
optimizer = optim.Adam(model_two_layers.parameters(), lr=0.001)

# Train the model with 50 epochs
num_epochs = 50
for epoch in range(num_epochs):
    model_two_layers.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model_two_layers(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/50], Loss: 0.3677
Epoch [20/50], Loss: 0.2931
Epoch [30/50], Loss: 0.2602
Epoch [40/50], Loss: 0.2388
Epoch [50/50], Loss: 0.2231


In [6]:
# Evaluation function to calculate accuracy
def evaluate_model(model, X_test_tensor, y_test_tensor):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predictions = model(X_test_tensor)
        predictions = predictions.round()  # Convert to 0 or 1
        accuracy = (predictions.eq(y_test_tensor).sum().item()) / len(y_test_tensor)
    return accuracy

# Evaluate both models
accuracy_one_layer = evaluate_model(model_one_layer, X_test_tensor, y_test_tensor)
accuracy_two_layers = evaluate_model(model_two_layers, X_test_tensor, y_test_tensor)

print(f'Accuracy with one hidden layer: {accuracy_one_layer:.4f}')
print(f'Accuracy with two hidden layers: {accuracy_two_layers:.4f}')

Accuracy with one hidden layer: 0.9031
Accuracy with two hidden layers: 0.9035
